In [2]:
# Import necessary modules
# glob allows unix style pathname pattern expansion
from glob import glob
# pydicom is the python dicom reader
import pydicom as dicom

In [14]:
# 5-T1w_MPR_vNav -- MPRAGE structrual dicoms
# 6-fMRI_DistortionMap_PA -- fmri field map dicoms
# 7-fMRI_DistortionMap_AP -- fmri field map dicoms opposite phase encode direction
# 9-fMRI_REVL_ROI_loc_2 -- fmri localizer dicoms run1
# 10-fMRI_REVL_Study_1 -- fmri task dicoms run1
# 14-dMRI_DistortionMap_AP_dMRI_REVL -- dwi field map dicoms
# 16-dMRI_AP_REVL -- diffusion weighted dicoms

# directories will have to be specific to your computer
base_dir = '/Users/shossein/Documents/000_PhD/Candidacy/fMRI_class/Homework'
proj_dir = ''
subj_dir = '/Mattfeld_REVL-000-vCAT-021-S1/scans/7-fMRI_DistortionMap_AP/resources/DICOM/files'

# with strings I can concatenate them with simple addition lines
data_dir = base_dir + proj_dir + subj_dir

# Here I use glob to grab the dicom files
# Why would I use glob? Because it is faster to retrieve all the files that are matching in their pathnames, as here.
# What does it give me? It gives a list of the pathnames to all the files in the "data_dir" as we used here.
data_files = glob(data_dir + '/*')


In [4]:
data_files

['/Users/shossein/Documents/000_PhD/Candidacy/fMRI_class/Homework/Mattfeld_REVL-000-vCAT-021-S1/scans/7-fMRI_DistortionMap_AP/resources/DICOM/files\\1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-7-1-17fcjdf.dcm',
 '/Users/shossein/Documents/000_PhD/Candidacy/fMRI_class/Homework/Mattfeld_REVL-000-vCAT-021-S1/scans/7-fMRI_DistortionMap_AP/resources/DICOM/files\\1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-7-10-1tc8ij0.dcm',
 '/Users/shossein/Documents/000_PhD/Candidacy/fMRI_class/Homework/Mattfeld_REVL-000-vCAT-021-S1/scans/7-fMRI_DistortionMap_AP/resources/DICOM/files\\1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-7-11-172hnkh.dcm',
 '/Users/shossein/Documents/000_PhD/Candidacy/fMRI_class/Homework/Mattfeld_REVL-000-vCAT-021-S1/scans/7-fMRI_DistortionMap_AP/resources/DICOM/files\\1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-7-12-40o5p.dcm',
 '/Users/shossein/Documents/000_PhD/Candidacy/fMRI_class/Homework/Mattfeld_REVL-000-vCAT-021-S1/sca

In [21]:
# Here I am using the python dicom reader to read in a dicom header
# What should go between the square brackets [] (Any number from 0 up to "the length of the data_set list -1" being 65 here)
data_set = dicom.dcmread(data_files[4])

In [22]:
data_set

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 204
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.2.43.166003.2019061915274255023814357
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.1
(0002, 0013) Implementation Version Name         SH: 'dcm4che-2.0'
(0002, 0016) Source Application Entity Title     AE: 'AN_MEDCOMNT204'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'ND']
(0008, 0012) Instance Creation Date              DA: '20190619'
(0008, 0013) Instance Creation Time              TM: '152745.328000'
(0008, 0016) SOP Class UID   

In [34]:
# Elements we're interested in
# Repetition Time
# Echo Time
# Acquisition Matrix 
# Flip Angle
# Acquisition Number
# Protocol Name
# Slice times = data_set[0x00191029].value

print('The number of TRs collected equals: {0}'.format(len(data_files)))
print('The timing of the TR was: {0} ms'.format(data_set.RepetitionTime))
print('The timing of the TE was: {0} ms'.format(data_set.EchoTime))
print('The flip angle in degrees was: {0}'.format(data_set.FlipAngle))
print('The name of the scan was: {0}'.format(data_set.ProtocolName))

# Why use the following format to access information in the header? The following 2 main reasons were discussed in the class: 
#1. Sometimes the information is private and we cannot acquire them using the given names, such as the previous ones. Sp we use these hexadecimal pointers.
#2. The hexadecimal pointers are always the same, and by knowing them, we can get the information from other DICOM files as well, regardless of how they name them.
print(data_set[int('00181312', 16)].value) #In-plane Phase Encoding Direction
print(data_set[int('0051100c', 16)].value) 
print(data_set[0x0051100c].value)


The number of TRs collected equals: 66
The timing of the TR was: 6720 ms
The timing of the TE was: 80 ms
The flip angle in degrees was: 90
The name of the scan was: fMRI_DistortionMap_AP
COL
FoV 200*200
FoV 200*200


In [44]:
# How do I find phase encoding direction information?
#First, from "In-plane Phase Encoding Direction" taged with (0018, 1312) we get that the Inplane phase ending is columnar, meaning that it is parallel to the sagittal plane.
#Second, by running the following code, we get access to the "CSA Image Header Info" and I think the item in"PhaseEncodingDirectionPositive" is 1, meaning that it is positive.
#So, the phase encoding direction would be from anterior to posterior (DICOM coordinate system).
import nibabel.nicom.csareader as csareader
csa_str = data_set[int('00291010', 16)].value
csa_tr = csareader.read(csa_str)
print(csa_tr['tags']['PhaseEncodingDirectionPositive']['items'][0])


1


In [41]:
csa_tr

{'tags': {'EchoLinePosition': {'n_items': 6,
   'vm': 1,
   'vr': 'IS',
   'syngodt': 6,
   'last3': 77,
   'tag_no': 0,
   'items': [50]},
  'EchoColumnPosition': {'n_items': 6,
   'vm': 1,
   'vr': 'IS',
   'syngodt': 6,
   'last3': 77,
   'tag_no': 1,
   'items': [50]},
  'EchoPartitionPosition': {'n_items': 6,
   'vm': 1,
   'vr': 'IS',
   'syngodt': 6,
   'last3': 77,
   'tag_no': 2,
   'items': [32]},
  'UsedChannelMask': {'n_items': 0,
   'vm': 1,
   'vr': 'UL',
   'syngodt': 9,
   'last3': 205,
   'tag_no': 3,
   'items': []},
  'UsedChannelString': {'n_items': 6,
   'vm': 1,
   'vr': 'UT',
   'syngodt': 27,
   'last3': 77,
   'tag_no': 4,
   'items': ['XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX']},
  'Actual3DImaPartNumber': {'n_items': 0,
   'vm': 1,
   'vr': 'IS',
   'syngodt': 6,
   'last3': 205,
   'tag_no': 5,
   'items': []},
  'ICE_Dims': {'n_items': 6,
   'vm': 1,
   'vr': 'LO',
   'syngodt': 19,
   'last3': 77,
   'tag_no': 6,
   'items': ['X_1_1_1_1_1_1_40_1_1_13_1_177']},
  '

In [43]:
print(csa_tr['tags']['PhaseEncodingDirectionPositive'])

{'n_items': 6, 'vm': 1, 'vr': 'IS', 'syngodt': 6, 'last3': 77, 'tag_no': 20, 'items': [1]}
